In [112]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector

import numpy as np

def quantum_fourier_transform(n):
    input_register = QuantumRegister(size=n, name="x")
    QFT_circuit = QuantumCircuit(input_register, name=f"QFT")

    for q, p in zip(input_register[:n >> 1], reversed(input_register[n >> 1:])):
        QFT_circuit.swap(q, p)

    for i, q in enumerate(input_register, start=1):
        QFT_circuit.h(q)
        for j, p in enumerate(input_register[i:], start=1):
            QFT_circuit.cp(np.pi / (1 << j), q, p)

    return QFT_circuit

def inverse_quantum_fourier_transform(n):
    input_register = QuantumRegister(size=n, name="x")
    inverse_QFT_circuit = QuantumCircuit(input_register, name=f"IQFT")

    for i, q in enumerate(reversed(input_register), start=1):
        for j, p in enumerate(reversed(input_register[n + 1 - i:]), start=1):
            inverse_QFT_circuit.cp(- np.pi / (1 << (i - j)), q, p)
        inverse_QFT_circuit.h(q)

    for q, p in zip(input_register[:n >> 1], reversed(input_register[n >> 1:])):
        inverse_QFT_circuit.swap(q, p)

    return inverse_QFT_circuit



<u> An implementation of the oracle for Shor's algorithm: </u> (mostly following Beauregard)

Task: given two inputs, $a,N \in \mathbb{Z}_{+}$, such that $a < N$, we construct a quantum circuit which implements the oracle:

$$
U | x \rangle_{1} | y \rangle_{n}  = \begin{cases}  | x \rangle_{1} | (a y) \text{ mod } N \rangle_{n} \text{, if } x = 1 \text{ and } y < N \\ 

|x \rangle_{1} |  y \rangle_{n} \text{, otherwise }

\end{cases}
$$

We note that this oracle should not have any effect if $y \geq  N$. With this extra condition, we will need to use an extra ancilla qubit in our construction to help keep track of the combination of the two conditions $x = 1$ <b> and </b> $ y < N$. That is, if both $x = 1$ and $y < N$ are true, then this extra ancilla will be in the $ | 1 \rangle$ state. Otherwise, the exta ancilla qubit will remain $| 0 \rangle$.  


If we pass in a state $| x \rangle_{1} | y \rangle_{n}$, such that $x = 1$ and $y < N$, then the output of our quantum circuit before cleaning ancillas, will be of the following form:

$$
| x \rangle_{1} \otimes | 0 \rangle_{1}  \otimes | \text{ extra ancilla } \rangle_{1} \otimes |   (a y) \text{ mod } N \rangle_{n} \otimes | 0 \rangle_{n}  =  | x \rangle_{1} \otimes | 0 \rangle_{1}  \otimes | 1 \rangle_{1} \otimes |   (a y) \text{ mod } N \rangle_{n} \otimes | 0 \rangle_{n}
$$

If either $x \neq 1$ or $y \geq  N$, then the output of our quantum circuit will be of the form:

$$
| x \rangle_{1} \otimes | 0 \rangle_{1}  \otimes | \text{ extra ancilla } \rangle_{1} \otimes |   y \rangle_{n} \otimes | 0 \rangle_{n}  =  | x \rangle_{1} \otimes | 0 \rangle_{1}  \otimes | 0 \rangle_{1} \otimes |  y \rangle_{n} \otimes | 0 \rangle_{n}
$$


In the end, we will be able to produce a construction with "clean ancillas". That is, the final output will be of the form:





$$ |x \rangle_{1} | y \rangle_{n} \mapsto 
 \begin{cases}  | x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | 0 \rangle_{1} \otimes  | (a y) \text{ mod } N \rangle_{n} \otimes | 0 \rangle_{n} \text{, if } x = 1 \text{ and } y < N \\ 

| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | 0 \rangle_{1} \otimes  | y \rangle_{n} \otimes | 0 \rangle_{n}  \text{, otherwise }
\end{cases}
$$

The construction presented in this notebook works for arbitrary $N$.

First, we write a function that outputs an adder circuit with optional multi-controls. It takes in parameters $a \in \mathbb{Z}$ and "num_of_controls" $\in \mathbb{N}$, where num_of_controls denotes the number of controls this adder should have. Then, the circuit performs the following (where $m = $ num_of_controls):

$$
| c_{1} , \cdots c_{m} \rangle_{m} | y \rangle_{n}  \mapsto | c_{1} , \cdots c_{m} \rangle_{m} | y + a \rangle_{n} 
$$

As usual, this adder is QFT based, and is essentially Draper's circuit with some optional controls.

In [113]:


def ControlledAdderCons(a, num_of_controls,n):
    input_register = QuantumRegister(n)
    
    if num_of_controls == 0:        
        ControlledAdder = QuantumCircuit(input_register) 
        ControlledAdder.compose(quantum_fourier_transform(n).to_gate(), inplace= True)    
        
        for q in enumerate(input_register):
            ControlledAdder.p( ( 2*np.pi / 2**n )*(2**q[0])*a, q[1]) 
            
                 
    else:
        control_register = QuantumRegister(num_of_controls)
        ControlledAdder = QuantumCircuit(input_register, control_register) 
        ControlledAdder.compose(quantum_fourier_transform(n).to_gate(), inplace= True) 
        for q in enumerate(input_register):
            ControlledAdder.mcp( ( 2*np.pi / 2**n )*(2**q[0])*a, control_register[:], q[1])
            
        
    ControlledAdder.compose(inverse_quantum_fourier_transform(n).to_gate(), inplace=True)
    return ControlledAdder
    
    

Next, we define a function that implements a <em> modular </em> addition circuit. It takes parameters $a, N \in \mathbb{Z}_{+}$, with $a < N$ and produces a circuit that acts as


$$
| y \rangle_{n} \mapsto | (y + a) \text{ mod } N \rangle_{n}
$$




We construct this circuit Following Beauregard. It is important here that $a < N$, otherwise the circuit may overflow and not result in the expected output. The circuit is constructed as follows:

1. Apply the adder circuit mapping $| y \rangle_{n} \mapsto | (y + a) \text{ mod } N \rangle_{n}$
2. Apply the inverse adder circuit (i.e. the subtractor circuit), which takes acts as $| x  \rangle_{n} \mapsto | x - N \rangle_{n}$. 

 As bitstrings encode integers in "twos complement" notation, one can check the state of the significant bit to see if $y + a - N$ is negative or not (i.e. whether or not $ y + a < N$). As both $y$ and $a$ are $< N$, we have two cases: 


$$
(y + a) \text{ mod } N = 
\begin{cases}
y + a - N \text{ , if } y + a \geq N \\
y + a \text{ , if } y + a < N 
\end{cases}
$$

3. Controlled on the significant-digit qubit (i.e. last qubit in the input register), we apply a CX gate to the qubit in the ancilla register. Therefore, at this stage, the ancilla qubit will flip from $| 0 \rangle$ to $| 1 \rangle$ iff $y + a < N$ 
4. Controlled on the ancilla qubit, we apply a controlled adder gate, which adds $N$ back to $y + a - N$, if $y + a - N < 0$.


At this stage, our qubit registers will be in state

$$
| (y + a) \text{ mod N } \rangle_{n} \otimes | \text{ dirty ancilla} = 0 \text{ or } 1 \rangle  \otimes | \text{ control qubits }   \rangle_{2}
$$

5. The last stage of the circuit is set up to clean up the dirty ancilla qubit and restore its state back to $| 0 \rangle$. 

The important observation here is that the ancilla will be in the state $| 1 \rangle$ iff $y + a < N$ iff $(y + a) \text{ mod } N  - a < y$. Therefore, by applying a subtractor circuit with the integer $a$, we can again check the significant digit qubit to see if this condition holds or not. That is,

6. We apply the inverse adder circuit mapping $| x  \rangle_{n} \mapsto | x - a \rangle_{n}$. 
7. After applying an X gate to the significant digit qubit of the input register, apply a CX gate controlled on this significant digit qubit targetting the ancilla qubit. The X gate flips the necessary condition from $y + a - N < 0$ to $y + a - N \geq  0$ (i.e. $y + a \geq N$).

At the end of the day, the circuit acts as:

$$
| y \rangle _{n} \mapsto | (y + a) \text{ mod } N \rangle_{n}
$$

with only clean ancilla qubits in the final output state.





In [114]:

def ControlledModAdderCons(a,N):
    n = N.bit_length() + 1
    input_register = QuantumRegister(n)
    anc_register = QuantumRegister(1)     
    control_register = QuantumRegister(2)
    
    
    
    ControlledModAdder = QuantumCircuit(input_register,control_register, anc_register)
    ControlledModAdder.compose(ControlledAdderCons(a,2,n), qubits= input_register[:] + control_register[:] , inplace=True)
    ControlledModAdder.compose(ControlledAdderCons(N,0,n).inverse(), qubits = input_register[:] , inplace=True)

    ControlledModAdder.cx(input_register[-1], anc_register[0])
    ControlledModAdder.compose(ControlledAdderCons(N,1,n), qubits= input_register[:] +  anc_register[:], inplace=True) 
    ControlledModAdder.compose(ControlledAdderCons(a,2,n).inverse(), qubits= input_register[:] + control_register[:], inplace=True)
    ControlledModAdder.x(input_register[-1])
    ControlledModAdder.cx(input_register[-1], anc_register[0])
    ControlledModAdder.x(input_register[-1])
    ControlledModAdder.compose(ControlledAdderCons(a,2,n ), qubits= input_register[:] + control_register[:], inplace=True)
    
    return ControlledModAdder





We will also need a function that outputs a controlled circuit implementing <em> modular multiplication </em>. The parameters here are again $a, N \in \mathbb{Z}_{+}$, with $a < N$. 
That is, we would like a circuit that acts as:

$$
| y \rangle_{n} \mapsto | (ay) \text{ mod } N \rangle_{n}
$$

The main observation here is that this circuit may be implemented by iterating a number of controlled modular adder gates. Consider the binary representation of $y = 2^{n-1} y_{n-1} + \cdots 2^{i} y_{i} + y_{0}$ where $y_{i} \in \{ 0, 1 \}$. Then, one has that 
$$(ay) \text{ mod } N =  $$
$$
(2^{n-1} a y_{n-1} + \cdots 2^{1} a y_{1} + 2^{0} a y_{0}) \text{ mod } N = 
$$
$$
(  \cdots (  ( ( ( a \cdot y_{0} ) \text{ mod } N ) + 2 \cdot  a \cdot y_{1} ) \text{ mod } N   + 2^{2} \cdot a \cdot y_{2} ) \text{ mod } N +  \cdots 2^{n-1 } \cdot a \cdot y_{n-1}  ) \text{ mod } N 
$$



The tricky part to this is that the modular adder gate specifically requires that we are adding an integer $a < N$ to the input register. However, there is no guarantee that $2^{k} \cdot a \cdot y_{k} < N$ just because $a < N$ and $y < N$. 

On the other hand, with the expression above, we only need to deal with modular addition with terms of the form $2^{k} \cdot a \cdot y_{k}$. To do this, we note that $a > N$ and $y_{k} > N$, and we can write:

$$(y + 2^{k} \cdot a \cdot y_{k} ) \text{ mod } N = $$
$$
 ( ( \cdots   ( ( ( y + a \cdot y_{k} ) \text{ mod } N ) + a \cdot y_{k} ) + a \cdot y_{k} ) \text{ mod } N  + \cdots ) 
$$

That is, we take the register $y$, and we iterate modular addition $2^{k}$ many times, where on each iteration we add $a \cdot y_{k} < N$. Now, as $a < N$, this can be implemented with the circuit we have constructed for modular addition.

In [115]:


def ControlledMultModCircuitCons(a,N):
    n = N.bit_length() + 1
    output_register = QuantumRegister(n)
    input_register = QuantumRegister(n)
    control_register = QuantumRegister(1)
    anc_register = QuantumRegister(1) 
            
    ControlledMultModCircuit = QuantumCircuit(output_register,input_register,control_register,anc_register)
    for q in enumerate(input_register):
        for i in range(2**q[0]):
            ControlledMultModCircuit.compose(ControlledModAdderCons(a , N ),  qubits=output_register[:]  + [q[1]]  + [control_register[0]] +  anc_register[:], inplace=True)
    return ControlledMultModCircuit
         

Note that in the above code, we need to initialize $2$ qubit registers of size $n$: one register will be named the "output register" and the other will be named the "input register". 

The initial state $| y \rangle_{n} = | y_{n-1} \cdots y_{0} \rangle_{n}$ will be encoded in the "input regster", and used for control qubits for modular addition by terms of the form 
$$
2^{k} \cdot a \cdot y_{k} = \begin{cases} 2^{k} \cdot a \text{, if } y_{k} = 1 \\ 0 \text{, otherwise } \end{cases}
$$


The output register is then used to encode the resulting states of the modular additions by each of the terms above. In the end, the state of the output register encodes the result of $(ay) \text{ mod } N$. Note that this means that the modular multiplication circuit constructed here is "out of place", in that the result of $(ay) \text{ mod } N$ is encoded in a different register than the register encoding the initial state $| y \rangle_{n}$. 


Explicitly, the circuit acts as follows (including ancilla and qubits): 

$$
| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | y \rangle_{n} \otimes  | 0 \rangle_{n} 
\mapsto
| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | y \rangle_{n} \otimes  | (a y) \text{ mod } N \rangle_{n} 
$$

as opposed to 

$$
| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | y \rangle_{n} \otimes  | 0 \rangle_{n} 
\mapsto
| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | (a y) \text{ mod } N \rangle_{n} \otimes  | 0 \rangle_{n} 
$$

----------------------------------------------------------------------------------


Finally, we come to the construction of the Shor oracle circuit. We write a function that takes in parameters $a , N \in \mathbb{Z}_{+}$ and outputs the desired Shor oracle circuit as described in the beginning of this notebook. That is, along with some clean ancillas, the circuit acts as:

$$ |x \rangle_{1} | y \rangle_{n} \mapsto 
 \begin{cases}  | x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | 0 \rangle_{1} \otimes  | (a y) \text{ mod } N \rangle_{n} \otimes | 0 \rangle_{n} \text{, if } x = 1 \text{ and } y < N \\ 

| x \rangle_{1}  \otimes | 0 \rangle_{1} \otimes | 0 \rangle_{1} \otimes  | y \rangle_{n} \otimes | 0 \rangle_{n} \text{, otherwise }
\end{cases}
$$

Note that this oracle circuit will perform a conditional modular multiplication operation "in place", in contrast to the "out of place" modular multiplication circuit we defined before.

We describe the construction of the circuit:

1. We initialize a register of size $2$, let us call this the ``oracle register". It will consist of $2$ qubits: one qubit is given by $| x \rangle$ (i.e. coming from the expected initial state of the oracle, which determines whether or not the circuit will act non-trivially). The other qubit is used as an ancilla qubit, whose state will be determined by the state of $| x \rangle$ and whether or not $y < N$. 
2. We also initialize a control register, consisting of a single qubit, which will be passed in as a control qubit for the modular multiplication circuits that we will use throughout this construction. These modular multiplication circuits will only operate if this control qubit is in state $|1 \rangle$.

3. The first block of code simply checks if $y < N$ and sets the state of ancilla qubit accordingly, controlled on the qubit $| x \rangle$. That is: we compose to the circuit, acting on the input register, the inverse of the controlled adder which represents subtraction by $N$. At this stage, the input register will be of the form $| y - N \rangle_{n}$. 

4. We apply a CX gate targetting the ancilla qubit, controlled on the qubit $| x \rangle$. Therefore, the ancilla qubit will flip from $| 0 \rangle$ to $| 1 \rangle $ iff $ | x \rangle  = | 1 \rangle$. 

5. We apply a CCX/Toffoli gate targetting the "control qubit" in the control register, controlled on the siginificant digit qubit of the input register and the ancilla qubit in the "oracle register". Then, the control qubit will flip from $| 0 \rangle$ to $| 1 \rangle$ iff $ x \rangle = | 1 \rangle$ and $y - N < 0$.

6. Then, we need to uncompute/clean the ancilla qubit in the oracle register, in order to have a "fresh ancilla" in the zero state for work in the rest of the circuit. This easy: as step $5$ doesn't affect the qubit $| x \rangle$ in the oracle register at all, we can uncompute the ancilla qubit by just reversing step $4$ (i.e. reapply it, as the gate in step $4$ is an involution)


7. Finally, we can compose to the circuit, our controlled modular multiplication circuit, controlled on the "control qubit" in the control register whose state was determined by steps $1$ to $6$. The modular multiplication (sub)-circuit will only operate if by the end of step $6$, the control qubit is in state $| 1 \rangle$.

8. The last stage of this circuit resolves the "out of place" issue present in the modular multiplication circuit. This "trick" is outlined in the Beauregard paper: given the state $| x \rangle_{n}  | 0 \rangle | 1 \rangle | y \rangle _{n} | (ay) \text{ mod } N \rangle_{n}$, we apply some swaps in order to switch around the output and input registers to achieve the state $| x \rangle_{n}  | 0 \rangle | 1 \rangle | (ay) \text{ mod } N \rangle_{n} | y \rangle _{n}$. Now, the qubits $| (ay) \text{ mod } N \rangle_{n}$ are in the input register and $| y \rangle_{n}$ is in the output register. 

We may pass this through the inverse of the modular multiplication circuit (controlled on the control qubit $| 1 \rangle$ in the oracle circuit). Since the input register now contains qubits $| (ay) \text{ mod } N \rangle_{n}$ and the output register contains $| y \rangle_{n}$, the effect of this inverse circuit will be:

$$
| x \rangle_{n}  | 0 \rangle | 1 \rangle | (ay) \text{ mod } N \rangle_{n} | y \rangle _{n} \mapsto | x \rangle_{n}  | 0 \rangle | 1 \rangle | (ay) \text{ mod } N \rangle_{n} | y - a^{-1} (ay)  \text{ mod } N \rangle _{n} = | x \rangle_{n}  | 0 \rangle | 1 \rangle | (ay) \text{ mod } N \rangle_{n} | 0 \rangle _{n}  
$$

Note that we will need the multiplicative inverse of $(a) \text{ mod } N$, to apply the inverse modular multiplication circuit as described above. This can be efficiently computed <em>classically </em> with the Euclidean algorithm, but we will just use a built-in python function to do this.


9. The final stage of this circuit is defined in order to uncompute the control qubit in the control register. The observation here is the following: 
- The control qubit is in state $|1 \rangle$ iff we had that $y < N$ and $x = 1$
- If $x = 1$, then input register will either be in the state $| (ay) \text{ mod } N \rangle_{n}$ or in the state $| y \rangle_{n}$. If it is in the state $| y \rangle_{n}$, then passing this through the subtractor circuit (subtract by $N$) will result in a $| 0 \rangle$ in the significant digit qubit. If it is in the state $| (ay) \text{ mod } N \rangle_{n}$, then the resulting state after subtraction will be in the state $| 1 \rangle$, as $(ay) \text{ mod } N < N$. 
- If $x = 0$, then there is no uncomputing to be done because if $x = 0$, then the control qubit will remain in the initial state of $ | 0 \rangle$.
Thus, to uncompute the control qubit, we apply compose with a controlled subtractor circuit (subtract by $N$), controlled on the qubit $|x \rangle$ in the oracle register, and as usual use a CCX-gate targetting the control qubit, controlled by the significant digit qubit in the input register, and the qubit $|x\rangle$ in the oracle register.




In [ ]:
def OracleCircuitCons(N, a):
   n = N.bit_length() + 1
   output_register = QuantumRegister(n)
   input_register = QuantumRegister(n)
   control_register = QuantumRegister(1)
   oracle_register = QuantumRegister(2) 
   
   # classical computation of a^{-1} modulo N.. i.e., the multiplicative inverse of (a) mod N this will be an integer less than N
   a_inv = pow(a, -1, N)
   
   OracleCircuit = QuantumCircuit(output_register, input_register, control_register, oracle_register)

   OracleCircuit.compose( ControlledAdderCons(N,0,n).inverse(), qubits= input_register[:], inplace=True )

   OracleCircuit.cx(oracle_register[1], oracle_register[0])
   OracleCircuit.ccx( input_register[-1], oracle_register[0], control_register[0])
   OracleCircuit.cx(oracle_register[1], oracle_register[0])

   OracleCircuit.compose( ControlledAdderCons(N,0,n), qubits= input_register[:], inplace=True )
   OracleCircuit.compose(ControlledMultModCircuitCons(a,N), qubits= output_register[:] + input_register[:] + control_register[:] + [oracle_register[0]], inplace=True)

   for i in range(n):
      OracleCircuit.cx(input_register[i],output_register[i])
      OracleCircuit.ccx(control_register[0], output_register[i], input_register[i])
      OracleCircuit.cx(input_register[i],output_register[i])

   OracleCircuit.compose(ControlledMultModCircuitCons(a_inv,N).inverse(), qubits = output_register[:]  + input_register[:]  + control_register[:] + [oracle_register[0]], inplace=True)

   OracleCircuit.compose( ControlledAdderCons(N,0,n).inverse(), qubits=input_register[:], inplace=True )
   OracleCircuit.ccx(oracle_register[1], input_register[-1], control_register[0])
   OracleCircuit.compose( ControlledAdderCons(N,0,n), qubits= input_register[:], inplace=True )


   return OracleCircuit

<u> <b> Testing </b> </u>

First, we test our code with $N = 15$

In [117]:
N = 15
n = N.bit_length() + 1


output_register = QuantumRegister(n)
oracle_register = QuantumRegister(2)

input_register = QuantumRegister(n, name='input')
control_register = QuantumRegister(1, name='ctrl')

We set $a = 7$, and initialize the input register to the state $| 7 \rangle_{n} = |  00 111 \rangle$ , and initialize the qubit $|x  \rangle$ to $| 1 \rangle$. Then, as $x = 1$ and $7 < 15$, the expected output should be

$$
| 1 \rangle \otimes | 0 0  \rangle \otimes  | 49 \text{ mod } N \rangle | 0 \rangle_{n} = | 1 \rangle \otimes | 0 0  \rangle \otimes  | 4  \rangle | 0 \rangle_{n} = | 1 \rangle \otimes | 0 0  \rangle \otimes  | 0 0 1 0 0 \rangle  \otimes | 0 \rangle_{n}  = | 1 0 0  0 0  1 0 0  0 0 0 0 0 \rangle
$$

In [118]:

a = 7

qc= QuantumCircuit(output_register,input_register, control_register, oracle_register)
qc.x( oracle_register[1])

qc.x(input_register[2])
qc.x(input_register[1])
qc.x(input_register[0])

qc.compose(OracleCircuitCons(N,a), inplace=True)



In [119]:
psi = Statevector(qc)

psi.draw('Latex')

<IPython.core.display.Latex object>

With the same example, we run this with the qubit $|x\rangle$ in the oracle register initialized to $| 0 \rangle$ instead of $| 1 \rangle$. As $| x \rangle = | 0 \rangle$, the expected output should be 

$$
| 0 \rangle \otimes | 0 0  \rangle \otimes  | 7 \text{ mod } N \rangle | 0 \rangle_{n} = | 0 \rangle \otimes | 0 0  \rangle \otimes  | 0 0 1 1 1  \rangle | 0 \rangle_{n} = | 0 0 0  0 0  1 1 1  0 0 0 0 0 \rangle
$$

In [120]:

qc= QuantumCircuit(output_register,input_register, control_register, oracle_register)

qc.x(input_register[2])
qc.x(input_register[1])
qc.x(input_register[0])

qc.compose(OracleCircuitCons(N,a), inplace=True)

psi = Statevector(qc)

psi.draw('Latex')

<IPython.core.display.Latex object>

Let us try initializing the input register to a state which encodes an integer larger than $N = 15$.  We set $| y \rangle_{n} = | 1 0 0 0 0 \rangle_{n} = | 16 \rangle$ and $| x \rangle = | 1 \rangle$. Then, the expected output is


$$
| 1 \rangle \otimes |0 \rangle |0 \rangle  \otimes  | 16 \rangle \otimes | 0 \rangle_{n}
= | 1 \rangle \otimes |0 \rangle |0 \rangle  \otimes  | 1 0 0 0 0 \rangle \otimes | 0 \rangle_{n} 
$$



In [121]:
qc= QuantumCircuit(output_register,input_register, control_register, oracle_register)
qc.x( oracle_register[1])

qc.x( input_register[4])

qc.compose(OracleCircuitCons(N,a), inplace=True)


In [122]:
psi = Statevector(qc)

psi.draw('Latex')

<IPython.core.display.Latex object>

The code runs as expected for any $N$, let us try $N = 21$.


In [123]:
a = 17 
N = 21
n = N.bit_length() + 1


output_register = QuantumRegister(n)
oracle_register = QuantumRegister(2)

input_register = QuantumRegister(n, name='input')
control_register = QuantumRegister(1, name='ctrl')

In [124]:
qc= QuantumCircuit(output_register,input_register, control_register, oracle_register)
qc.x( oracle_register[1])

qc.x(input_register[3])
qc.x(input_register[2])
qc.x(input_register[0])

qc.compose(OracleCircuitCons(N,a), inplace=True)


In [125]:
psi = Statevector(qc)

psi.draw('Latex')

<IPython.core.display.Latex object>

Here is an example with $N = 29$. We set $a = 21$ and initialize $| x \rangle = | 1 \rangle$ and $| y \rangle = | 7 \rangle =  | 0 0 0 1 1 1 \rangle$. Thus, the expected output should be

$$
| 1 \rangle \otimes | 0 0 \rangle  \otimes | 7 \cdot 21 \text{ mod } 29 \rangle_{n} \otimes | 0 \rangle_{n} = | 1 \rangle \otimes | 0 0 \rangle  \otimes | 2 \rangle_{n} \otimes | 0 \rangle_{n} = | 1 \rangle \otimes | 0 0 \rangle  \otimes | 0 0 0 0 1 0  \rangle_{n} \otimes | 0 \rangle_{n} = | 1  0 0  0 0  0  0  1 0 0 0 0 0 0 0 \rangle
$$

In [127]:

a = 21 
N = 29
n = N.bit_length() + 1


output_register = QuantumRegister(n)
oracle_register = QuantumRegister(2)

input_register = QuantumRegister(n, name='input')
control_register = QuantumRegister(1, name='ctrl')
qc= QuantumCircuit(output_register,input_register, control_register, oracle_register)
qc.x( oracle_register[1])

qc.x(input_register[2])
qc.x(input_register[1])
qc.x(input_register[0])

qc.compose(OracleCircuitCons(N,a), inplace=True)

psi = Statevector(qc)

psi.draw('Latex')

<IPython.core.display.Latex object>